In [ ]:
from cobraCoach import CobraCoach
import pathlib
import numpy as np
import logging
import engineer as eng

In [ ]:
cc = CobraCoach('fpga', loadModel=False)
cc.loadModel(version='SC32', moduleVersion=3)
eng.setCobraCoach(cc)

In [ ]:
# update center positions from home, not accurately
eng.convertXML('SC32_init.xml')

In [ ]:
cc.showStatus()

In [ ]:
# enable motor scaling and set parameters
cc.setScaling(True, thetaScaleFactor=2.0, phiScaleFactor=2.0, minThetaSteps=10, minPhiSteps=10)

In [ ]:
# reset motor scaling
cc.pfi.resetMotorScaling()

In [ ]:
# switch to phi mode
eng.setPhiMode()

In [ ]:
# generate a quick motormap and set geometry
eng.makePhiMotorMaps('phi150.xml', steps=150, totalSteps=6000, repeat=1)

In [ ]:
# set phi geometry from previous run
cc.setPhiGeometryFromRun('/data/MCS/20200520_045', onlyIfClear=False)

In [ ]:
# move phi to home position
cc.moveToHome(cc.goodCobras, phiEnable=True)

In [ ]:
# phi on time search
eng.phiOnTimeSearch('SC32_phi_search.xml', iteration=3, repeat=3)

In [ ]:
# generate new motormaps
eng.makePhiMotorMaps('SC32_phi800.xml', steps=800, totalSteps=8000, repeat=3)
eng.makePhiMotorMaps('SC32_phi500.xml', steps=500, totalSteps=8000, repeat=3)
eng.makePhiMotorMaps('SC32_phi200.xml', steps=200, totalSteps=8000, repeat=3)
eng.makePhiMotorMaps('SC32_phi100.xml', steps=100, totalSteps=8000, repeat=3)
eng.makePhiMotorMaps('SC32_phi50.xml', steps=50, totalSteps=8000, repeat=3)
eng.makePhiMotorMaps('SC32_phi20.xml', steps=20, totalSteps=8000, repeat=3)
eng.makePhiMotorMaps('SC32_phi10.xml', steps=10, totalSteps=8000, repeat=3)
eng.makePhiMotorMaps('SC32_phi5.xml', steps=5, totalSteps=6000, repeat=3)

In [ ]:
# mvoe phi by angles
dataPath, diffAngles, moves = eng.movePhiAngles(cc.goodIdx, np.deg2rad(165), relative=False, local=True, tolerance=0.002, tries=6, fast=False, newDir=True)

In [ ]:
# move by a angle
dataPath, diffAngles, moves = eng.movePhiAngles(cc.goodIdx, np.deg2rad(90), relative=False, local=True, tolerance=0.002, tries=6, fast=False, newDir=True)

In [ ]:
# convergence test
moves = eng.phiConvergenceTest(cc.goodIdx, margin=15.0, runs=2, tries=8, fast=False, tolerance=0.1)

In [ ]:
# move phi to 60 degree for theta test
dataPath, diffAngles, moves = eng.movePhiAngles(cc.goodIdx, np.pi/3, relative=False, local=False, tolerance=0.002, tries=6, fast=False, newDir=True)

In [ ]:
# move phi arms to 60 degree (relative to theta arms) in phi mode
eng.movePhiAngles(cc.goodIdx, np.pi/3, local=False)

In [ ]:
# swtich to theta mode
eng.setThetaMode()

In [ ]:
# generate a quick motormap and set geometry
eng.makeThetaMotorMaps('theta250.xml', steps=250, totalSteps=10000, repeat=1)

In [ ]:
# set theta geometry from previous run
cc.setThetaGeometryFromRun('/data/MCS/20200504_019', onlyIfClear=False)

In [ ]:
# move theta to home position
cc.moveToHome(cc.goodCobras, thetaEnable=True)

In [ ]:
# set current angle to zeros, if you know
cc.setCurrentAngles(cc.goodCobras, thetaAngles=0, phiAngles=np.pi/3)

In [ ]:
# theta on time search
eng.thetaOnTimeSearch('SC32_theta_search.xml', iteration=5, repeat=3)

In [ ]:
# generate new motormaps
eng.makeThetaMotorMaps('SC32_theta800.xml', steps=800, totalSteps=12000, repeat=3)
eng.makeThetaMotorMaps('SC32_theta500.xml', steps=500, totalSteps=12000, repeat=3)
eng.makeThetaMotorMaps('SC32_theta200.xml', steps=200, totalSteps=12000, repeat=3)
eng.makeThetaMotorMaps('SC32_theta100.xml', steps=100, totalSteps=12000, repeat=3)
eng.makeThetaMotorMaps('SC32_theta50.xml', steps=50, totalSteps=12000, repeat=3)

In [ ]:
eng.makeThetaMotorMaps('SC32_theta20.xml', steps=20, totalSteps=12000, repeat=3, force=True)
eng.makeThetaMotorMaps('SC32_theta10.xml', steps=10, totalSteps=12000, repeat=3, force=True)

In [ ]:
eng.makeThetaMotorMaps('SC32_theta5.xml', steps=5, totalSteps=12000, repeat=3, force=True)

In [ ]:
# reset motor scaling
cc.pfi.resetMotorScaling(cc.allCobras)

In [ ]:
# move by angles
dataPath, diffAngles, moves = eng.moveThetaAngles(cc.goodIdx, np.pi/2, tries=6)

In [ ]:
# convergence test
moves = eng.thetaConvergenceTest(cc.goodIdx, margin=15.0, runs=3, tries=8, fast=False, tolerance=0.1)

In [ ]:
# move theta arms upward in theta mode
eng.moveThetaAngles(cc.goodIdx, np.pi/2, local=False)

In [ ]:
# update cobra geometry
eng.updateXML('SC32_02.xml', '/data/MCS/20200520_013', '/data/MCS/20200520_009')

In [ ]:
# swtich to normal mode
eng.setNormalMode()

In [ ]:
# move to home position
cc.moveToHome(cc.goodCobras, thetaEnable=True, phiEnable=True, thetaCCW=False)

In [ ]:
# set current angle, if you know
cc.setCurrentAngles(cc.goodCobras, thetaAngles=0, phiAngles=0)

In [ ]:
# move by angles
dataPath, thetas, phis, moves = eng.moveThetaPhi(cc.goodIdx, np.pi/2, np.pi/3, relative=False, local=True, tolerance=0.1, tries=6, homed=True, newDir=True)

In [ ]:
# move by positions, go home first and do safe movement
tries = 12
targets = cc.pfi.anglesToPositions(cc.allCobras, np.full(cc.nCobras, np.pi), np.full(cc.nCobras, np.pi/2))
dataPath, positions, moves = eng.movePositions(cc.goodIdx, targets[cc.goodIdx], tolerance=0.1, tries=tries, thetaMarginCCW=0.1, homed=True, newDir=True)

In [ ]:
# move by positions again
targets = cc.pfi.anglesToPositions(cc.allCobras, np.full(cc.nCobras, np.pi), np.full(cc.nCobras, np.pi/2))
dataPath, positions, moves = eng.movePositions(cc.goodIdx, targets[cc.goodIdx], tolerance=0.1, tries=8, thetaMarginCCW=0.1, homed=False, newDir=True)

In [ ]:
targets, moves = eng.convergenceTest(cc.goodIdx, iteration=1, thetaMargin=np.deg2rad(10.0), phiMargin=np.deg2rad(25.0), tries=8, tolerance=0.2)

In [ ]:
# goto safe position
tht = np.empty(cc.nCobras)
tht[0::2] = np.pi*3/2
tht[1::2] = np.pi/2
dataPath, thetas, phis, moves = eng.moveThetaPhi(cc.goodIdx, tht[cc.goodIdx], np.pi/3, relative=False, local=False, tolerance=0.1, tries=6, homed=False, newDir=True)

In [ ]:
cc.showStatus()

In [ ]:
targets[0,0], moves[0,0], targets[0,35], moves[0,35], targets[0,37], moves[0,37], targets[0,53], moves[0,53]

In [ ]:
moves

In [ ]:
targets

In [ ]:
cc.moveInfo

In [ ]:
cc.pfi.calibModel.cobraStatus(31)

In [ ]:
cc.pfi.calibModel.setCobraStatus(31, brokenTheta=True)
cc.pfi.calibModel.createCalibrationFile('SC32_3.xml')